In [1]:
# standard import
from datetime import date
import datetime
import pandas as pd
import numpy as np
import os
import glob
import PyPDF2
import docx

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


os.chdir('/Users/jiayikoh/KJY/IPP')

# state the folder_path
folder_path = '/Users/jiayikoh/Downloads/Dec 23 (Payout in Jan 2024)'

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
# Read all sheets into a dictionary of dataframes
uob_cb_dict = pd.read_excel('/Users/jiayikoh/Downloads/Dec 23 (Payout in Jan 2024)/IPP-FA-UOB SGD Yr 2023.xlsx', sheet_name=None, header=4)

# Concatenate all dataframes into one
uob_cb = pd.concat(uob_cb_dict.values(), ignore_index=True)


# Read all sheets into a dictionary of dataframes
rhb_cb_dict = pd.read_excel('/Users/jiayikoh/Downloads/Dec 23 (Payout in Jan 2024)/IPP-FA-RHB SGD Yr 2023.xlsx', sheet_name=None, header=4)

# Concatenate all dataframes into one
rhb_cb = pd.concat(rhb_cb_dict.values(), ignore_index=True)


# extract the insurer name
uob_cb['Extracted_Insurer'] = uob_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()
rhb_cb['Extracted_Insurer'] = rhb_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()

# concatenate both cashbook df
all_cb = pd.concat([uob_cb, rhb_cb], axis=0)

In [3]:
# to load the current P0 working file to get the previous information for the missing TFAR in some of the insurer's files
wk_dict = pd.read_excel('/Users/jiayikoh/Downloads/Dec 23 (Payout in Jan 2024)/P0.12 Dec 23 (Payout in Jan 2024).xlsx', sheet_name=None)

# define the keys to be concatenate
paid_keys = [key for key in wk_dict.keys() if key.startswith('PAID')]
unpaid_keys = [key for key in wk_dict.keys() if key.startswith('UNPAID')]

# concatenate both
all_keys = paid_keys + unpaid_keys

# concatenate the corresponding dataframe
previous_wk = pd.concat([wk_dict[key] for key in all_keys]).drop_duplicates()

In [4]:
from gi_test import GI_commission

In [5]:
from Name_matching import Name_matching

In [6]:
gi = GI_commission(folder_path=folder_path, all_cb=all_cb, previous_wk=previous_wk)

In [7]:
# concatenate all the df
wk_df_1 = pd.concat([gi.run_aig_1('01_01A'),
           gi.run_aig_1('01_01B'),
           gi.run_aia_1('03_01A'),
           gi.run_allianz_1('04_01A'),
           gi.run_allied_1('06_01A'),
           gi.run_allied_1('06_01B'),
           gi.run_allied_2('06_02A'),
           gi.run_chubb_1('09_01A'),
           gi.run_chubb_1('09_01B'),
           gi.run_ergo_1('13_01A'),
           gi.run_fwd_1('15_01A'),
           gi.run_ge_life_1('18_01A'),
           gi.run_hla_1('19_01A'),
           gi.run_hsbc_1('20_01A'),
           gi.run_income_1('21_01A'),
           gi.run_liberty_1('23_01A'),
           gi.run_liberty_1('23_01B'),
           gi.run_liberty_1('23_01C'),
           gi.run_liberty_1('23_01D'),
           gi.run_nhi_1('26_01A'),
           gi.run_qbe_1('28_01A'),
           gi.run_singlife_1('30_01A'),
           gi.run_sompo_1('31_01A'),
           gi.run_sompo_1('31_01B')
           ], ignore_index=True).drop_duplicates()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [8]:
name_far = Name_matching(folder_path=folder_path, all_advisor_df=wk_df_1)

In [9]:
comm_far = name_far.matching_far()

In [ ]:
# read the FAR masterlist
file_pattern = 'W1*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

far_master = pd.read_excel(matching_files[0], skiprows=1)

# merge with itself to get the old agent's manager details
far_master = pd.merge(far_master, far_master[['NEW FAR NAME', 'Manager', 'Mgr Rate']].rename(columns={'Manager':'Manager 2 Name', 'Mgr Rate':'Manager 2 GR%'}),
                     how='left', left_on='FAR (Old)', right_on='NEW FAR NAME').rename(columns={'NEW FAR NAME_x':'NEW FAR NAME'})

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_2 = pd.merge(wk_df_1, comm_far[['ADVISER', 'matched_name']], how='left', left_on='TFAR', right_on='ADVISER')

# match the adviser details using the name matching list
wk_df_3 = pd.merge(wk_df_2, far_master[['NEW FAR NAME', 'Advisory Group', 2023, 'FAR Status']].rename(columns={2023:'GR%'}), how='left', left_on='matched_name', right_on='NEW FAR NAME')

# match policy number to get referrer details
wk_df_4 = pd.merge(wk_df_3, e_sub[['Policy No', 'Name of Referral']].rename(columns={'Name of Referral':'Referrer'}), how='left', left_on='Policy no.', right_on='Policy No').drop_duplicates()

# remove Not Applicable in Referrer
wk_df_4['Referrer'] = np.where(wk_df_4['Referrer'] == 'Not Applicable', np.NaN, wk_df_4['Referrer'])

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_5 = pd.merge(wk_df_4, comm_far[['ADVISER', 'matched_name']].rename(columns={'matched_name':'referrer_matched_name'}), how='left', left_on='Referrer', right_on='ADVISER')


# label GST Type
insurer_list = list(wk_df_5['Insurer'].unique())

new_df = []

for i, j in enumerate(insurer_list):
    df_working = wk_df_5[wk_df_5['Insurer'] == j]
    if j == 'AIG-GI':
        new_df.append(df_working)
    elif j == 'AIA-GI':
        df_working['GST Type'] = np.where(df_working['Sum of GST'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIED WORLD-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIANZ-GI':
        df_working['GST Type'] = np.where(df_working['GST on Commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'CHUBB-GI':
        new_df.append(df_working)
        #df_working['GST Type'] = np.where(df_working['Sum of GST'] == 0, 'Z', 'G')
    elif j == 'FWD-GI':
        df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'GE-GI':
        new_df.append(df_working)
        #df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
    elif j == 'HSBC-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'INCOME-GI':
        df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'INDIA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'LIBERTY-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'MSIG-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'QBE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SINGLIFE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SOMPO-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'V')
        new_df.append(df_working)
        
        
wk_df_6 = pd.concat(new_df)
    

# make a soft copy
wk_df = wk_df_6.drop_duplicates().copy()

# delete TFAR column
del wk_df['TFAR']
del wk_df['Referrer']

# get the required fields
columns = ['Cashbook ref. no.', 'Policy no.', 'Insured Name', 'Insurer', 'TFAR', 'Referrer', 'Advisory Group', 'Comm.Recd (with GST)', 'GST Type', 'GR%', 'Pol Date', 'JY_comment', 'FAR Receiving Comm Status']

wk_df = wk_df.rename(columns={'Advisory Group_y':'Advisory Group', 'matched_name':'TFAR', 'referrer_matched_name':'Referrer', 'GR%_y':'GR%', 'FAR Status':'FAR Receiving Comm Status'})[columns].drop_duplicates()

# title case for Insured Name and Advisory Group
#wk_df['Insured Name'] = wk_df['Insured Name'].str.title()
#wk_df['Advisory Group'] = wk_df['Advisory Group'].str.title()

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{folder_path}Working_file_{date.today():%Y%m%d}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
wk_df.to_excel(writer, sheet_name='Data')
#df_1_pivot.to_excel(writer, sheet_name='Pivot')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### below is the final script for GI Commission

In [ ]:
# standard import
from datetime import date
import datetime
import pandas as pd
import numpy as np
import os
import glob
import PyPDF2
import docx

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

folder_path = input("Please input the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")

while os.path.isdir(folder_path) != True:
    folder_path = input("Directory does not exist, please re-enter the directory to the files -> /Users/jiayikoh/Downloads/07 GI Commission 3/\n")
os.chdir(folder_path)

uob_cashbook = input("UOB cashbook file name\n")

rhb_cashbook = input("RHB cashbook file name\n")

# load the class for all insurer functions
from gi_test import GI_commission

# load the class for name matching function
from Name_matching import Name_matching


# Read all sheets into a dictionary of dataframes
uob_cb_dict = pd.read_excel(uob_cashbook, sheet_name=None, header=4)

# Concatenate all dataframes into one
uob_cb = pd.concat(uob_cb_dict.values(), ignore_index=True)

# Read all sheets into a dictionary of dataframes
rhb_cb_dict = pd.read_excel(rhb_cashbook, sheet_name=None, header=4)

# Concatenate all dataframes into one
rhb_cb = pd.concat(rhb_cb_dict.values(), ignore_index=True)


# extract the insurer name
uob_cb['Extracted_Insurer'] = uob_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()
rhb_cb['Extracted_Insurer'] = rhb_cb['Name'].str.extract('([^\s]+)', expand=False).str.lower()

# concatenate both cashbook df
all_cb = pd.concat([uob_cb, rhb_cb], axis=0)


# to load the current P0 working file to get the previous information for the missing TFAR in some of the insurer's files
wk_dict = pd.read_excel('/Users/jiayikoh/Downloads/Dec 23 (Payout in Jan 2024)/P0.12 Dec 23 (Payout in Jan 2024).xlsx', sheet_name=None)

# define the keys to be concatenate
paid_keys = [key for key in wk_dict.keys() if key.startswith('PAID')]
unpaid_keys = [key for key in wk_dict.keys() if key.startswith('UNPAID')]

# concatenate both
all_keys = paid_keys + unpaid_keys

# concatenate the corresponding dataframe
previous_wk = pd.concat([wk_dict[key] for key in all_keys]).drop_duplicates()



# instantiate the GI Commission class
gi = GI_commission(folder_path=folder_path, all_cb=all_cb, previous_wk=previous_wk)


# concatenate all the df
wk_df_1 = pd.concat([gi.run_aig_1('01_01A'),
           gi.run_aig_1('01_01B'),
           gi.run_aia_1('03_01A'),
           gi.run_allianz_1('04_01A'),
           gi.run_allied_1('06_01A'),
           gi.run_allied_1('06_01B'),
           gi.run_allied_2('06_02A'),
           gi.run_chubb_1('09_01A'),
           gi.run_chubb_1('09_01B'),
           gi.run_ergo_1('13_01A'),
           gi.run_fwd_1('15_01A'),
           gi.run_ge_life_1('18_01A'),
           gi.run_hla_1('19_01A'),
           gi.run_hsbc_1('20_01A'),
           gi.run_income_1('21_01A'),
           gi.run_liberty_1('23_01A'),
           gi.run_liberty_1('23_01B'),
           gi.run_liberty_1('23_01C'),
           gi.run_liberty_1('23_01D'),
           gi.run_nhi_1('26_01A'),
           gi.run_qbe_1('28_01A'),
           gi.run_singlife_1('30_01A'),
           gi.run_sompo_1('31_01A'),
           gi.run_sompo_1('31_01B')
           ], ignore_index=True).drop_duplicates()


# instantiate the name matching class
name_far = Name_matching(folder_path=folder_path, all_advisor_df=wk_df_1)

# get the matched FAR name
comm_far = name_far.matching_far()


# read the FAR masterlist
file_pattern = 'W1*.xls*' 

matching_files = glob.glob(os.path.join(folder_path, file_pattern))

far_master = pd.read_excel(matching_files[0], skiprows=1)

# merge with itself to get the old agent's manager details
far_master = pd.merge(far_master, far_master[['NEW FAR NAME', 'Manager', 'Mgr Rate']].rename(columns={'Manager':'Manager 2 Name', 'Mgr Rate':'Manager 2 GR%'}),
                     how='left', left_on='FAR (Old)', right_on='NEW FAR NAME').rename(columns={'NEW FAR NAME_x':'NEW FAR NAME'})

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_2 = pd.merge(wk_df_1, comm_far[['ADVISER', 'matched_name']], how='left', left_on='TFAR', right_on='ADVISER')

# match the adviser details using the name matching list
wk_df_3 = pd.merge(wk_df_2, far_master[['NEW FAR NAME', 'Advisory Group', 2023, 'FAR Status']].rename(columns={2023:'GR%'}), how='left', left_on='matched_name', right_on='NEW FAR NAME')

# match policy number to get referrer details
wk_df_4 = pd.merge(wk_df_3, e_sub[['Policy No', 'Name of Referral']].rename(columns={'Name of Referral':'Referrer'}), how='left', left_on='Policy no.', right_on='Policy No').drop_duplicates()

# remove Not Applicable in Referrer
wk_df_4['Referrer'] = np.where(wk_df_4['Referrer'] == 'Not Applicable', np.NaN, wk_df_4['Referrer'])

# find the name in FAR masterlist using the revised name list provided by the name matching algorithm
wk_df_5 = pd.merge(wk_df_4, comm_far[['ADVISER', 'matched_name']].rename(columns={'matched_name':'referrer_matched_name'}), how='left', left_on='Referrer', right_on='ADVISER')


# label GST Type
insurer_list = list(wk_df_5['Insurer'].unique())

new_df = []

for i, j in enumerate(insurer_list):
    df_working = wk_df_5[wk_df_5['Insurer'] == j]
    if j == 'AIG-GI':
        new_df.append(df_working)
    elif j == 'AIA-GI':
        df_working['GST Type'] = np.where(df_working['Sum of GST'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIED WORLD-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'ALLIANZ-GI':
        df_working['GST Type'] = np.where(df_working['GST on Commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'CHUBB-GI':
        new_df.append(df_working)
        #df_working['GST Type'] = np.where(df_working['Sum of GST'] == 0, 'Z', 'G')
    elif j == 'FWD-GI':
        df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'GE-GI':
        new_df.append(df_working)
        #df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
    elif j == 'HSBC-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'INCOME-GI':
        df_working['GST Type'] = np.where(df_working['$txn gst commission'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'INDIA-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'LIBERTY-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'MSIG-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'QBE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SINGLIFE-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'G')
        new_df.append(df_working)
    elif j == 'SOMPO-GI':
        df_working['GST Type'] = np.where(df_working['GST amt'] == 0, 'Z', 'V')
        new_df.append(df_working)
        
        
wk_df_6 = pd.concat(new_df)
    

# make a soft copy
wk_df = wk_df_6.drop_duplicates().copy()

# delete TFAR column
del wk_df['TFAR']
del wk_df['Referrer']

# get the required fields
columns = ['Cashbook ref. no.', 'Policy no.', 'Insured Name', 'Insurer', 'TFAR', 'Referrer', 'Advisory Group', 'Comm.Recd (with GST)', 'GST Type', 'GR%', 'Pol Date', 'JY_comment', 'FAR Receiving Comm Status']

wk_df = wk_df.rename(columns={'Advisory Group_y':'Advisory Group', 'matched_name':'TFAR', 'referrer_matched_name':'Referrer', 'GR%_y':'GR%', 'FAR Status':'FAR Receiving Comm Status'})[columns].drop_duplicates()

# title case for Insured Name and Advisory Group
#wk_df['Insured Name'] = wk_df['Insured Name'].str.title()
#wk_df['Advisory Group'] = wk_df['Advisory Group'].str.title()

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(f'{folder_path}Working_file_{date.today():%Y%m%d}.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
wk_df.to_excel(writer, sheet_name='Data')
#df_1_pivot.to_excel(writer, sheet_name='Pivot')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [11]:
!pip freeze > requirements.txt